In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor




from sklearn.decomposition import PCA

In [63]:
df=pd.read_csv("gurgaon_properties_post_feature_selection_v2.csv")

In [66]:
df.head(3)

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor


In [85]:
df.shape

(3554, 13)

In [67]:
"""
X = df.drop(columns=['price'])
y = df['price']
columns_to_encode = ["property_type",'sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']
# Applying the log1p transformation to the target variable, because it was right skewed
y_transformed = np.log1p(y)
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer([
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), columns_to_encode)  # Use OneHotEncoder directly
    ], remainder='passthrough')

# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
]) 

# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
print(scores.mean())
print(scores.std())
"""

'\nX = df.drop(columns=[\'price\'])\ny = df[\'price\']\ncolumns_to_encode = ["property_type",\'sector\', \'balcony\', \'agePossession\', \'furnishing_type\', \'luxury_category\', \'floor_category\']\n# Applying the log1p transformation to the target variable, because it was right skewed\ny_transformed = np.log1p(y)\n# Creating a column transformer for preprocessing\npreprocessor = ColumnTransformer([\n        (\'num\', StandardScaler(), [\'property_type\', \'bedRoom\', \'bathroom\', \'built_up_area\', \'servant room\', \'store room\']),\n        (\'cat\', OneHotEncoder(drop=\'first\', handle_unknown=\'ignore\'), columns_to_encode)  # Use OneHotEncoder directly\n    ], remainder=\'passthrough\')\n\n# Creating a pipeline\npipeline = Pipeline([\n    (\'preprocessor\', preprocessor),\n    (\'regressor\', SVR(kernel=\'rbf\'))\n]) \n\n# K-fold cross-validation\nkfold = KFold(n_splits=10, shuffle=True, random_state=42)\nscores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring=\'

In [68]:
"""
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

"""

'\nX_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)\npipeline.fit(X_train,y_train)\ny_pred = pipeline.predict(X_test)\ny_pred = np.expm1(y_pred)\nfrom sklearn.metrics import mean_absolute_error\nmean_absolute_error(np.expm1(y_test),y_pred)\n\n'

In [69]:
df['furnishing_type'].value_counts()

furnishing_type
0.0    2349
1.0    1018
2.0     187
Name: count, dtype: int64

In [70]:
#for end user we will change the numerical values in the category to following
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [71]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor


In [78]:
X = df.drop(columns=['price'])
y = df['price']

In [79]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

### 1. Ordinal Encoding

In [80]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [81]:
# Column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [82]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [83]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [86]:
scores.mean(),scores.std()

(0.7363096633436828, 0.03238005754429931)

Theses scores are not very good coz we are applying linear model on ordinal encoded data which can reduce the performance of the model.

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [88]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [89]:
y_pred = pipeline.predict(X_test)

In [90]:
y_pred = np.expm1(y_pred)

In [91]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9463822160089367

In [92]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [93]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [94]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [95]:
model_output

[['linear_reg', 0.7363096633436828, 0.9463822160089367],
 ['svr', 0.7642079133544131, 0.8472636473483923],
 ['ridge', 0.7363125343993554, 0.9463387741853377],
 ['LASSO', 0.059433780644935694, 1.5289059868927528],
 ['decision tree', 0.7685715732617743, 0.7447431468969506],
 ['random forest', 0.8803779624126499, 0.5288035907653653],
 ['extra trees', 0.8676052448561531, 0.5455115925521355],
 ['gradient boosting', 0.8726335630265014, 0.5763596048481858],
 ['adaboost', 0.7566546502943796, 0.8179360776416662],
 ['mlp', 0.8090347978835855, 0.6846786834189375],
 ['xgboost', 0.8894876835260124, 0.5040475180464287]]

In [96]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [97]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.889488,0.504048
5,random forest,0.880378,0.528804
6,extra trees,0.867605,0.545512
7,gradient boosting,0.872634,0.576360
9,mlp,0.809035,0.684679
4,decision tree,0.768572,0.744743
8,adaboost,0.756655,0.817936
1,svr,0.764208,0.847264
2,ridge,0.736313,0.946339
0,linear_reg,0.736310,0.946382


# Observations:
1. With Ordinal encoding tree based models gave good results and linear models gave unsatisfactory results. 

### One Hot Encoding

In [73]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [98]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [99]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [100]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [101]:
scores.mean(),scores.std()

(0.8546079524302987, 0.015997658281575768)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [103]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'agePossession',
                                                   'furnishing_type'])])),
                ('regressor', LinearRegression())])

In [104]:
y_pred = pipeline.predict(X_test)

In [105]:
y_pred = np.expm1(y_pred)

In [106]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.6497670385884241

In [107]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [108]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [109]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [110]:
model_output

[['linear_reg', 0.8546079524302987, 0.6497670385884241],
 ['svr', 0.7697413260547326, 0.8341243500492123],
 ['ridge', 0.8546483034406196, 0.652814159032892],
 ['LASSO', 0.05943378064493573, 1.528905986892753],
 ['decision tree', 0.8051616071934985, 0.698336602346718],
 ['random forest', 0.8904091791038656, 0.5122124983007346],
 ['extra trees', 0.8938441508474442, 0.46934330217423004],
 ['gradient boosting', 0.8766353281238992, 0.5702045443760484],
 ['adaboost', 0.7519856542438058, 0.8367707917755443],
 ['mlp', 0.8750418203163897, 0.5305954849458226],
 ['xgboost', 0.8958499681743852, 0.4934562632085233]]

In [111]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [112]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.893844,0.469343
10,xgboost,0.895850,0.493456
5,random forest,0.890409,0.512212
9,mlp,0.875042,0.530595
7,gradient boosting,0.876635,0.570205
0,linear_reg,0.854608,0.649767
2,ridge,0.854648,0.652814
4,decision tree,0.805162,0.698337
1,svr,0.769741,0.834124
8,adaboost,0.751986,0.836771


### OneHotEncoding With PCA

In [28]:
columns_to_encode1 = ['property_type','balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [113]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [114]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [115]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [116]:
scores.mean()

0.06225201431451146

1. Here scores reduced surprisingly, we need to inspect why this happened.
2. We will not consider applying PCA without finding out the cause of reduction in the scores. 


In [117]:
scores.std()

0.01986059407164007

In [118]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [119]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [120]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [121]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [122]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.763474,0.652781
6,extra trees,0.739186,0.703178
4,decision tree,0.696182,0.757290
10,xgboost,0.620664,0.948597
7,gradient boosting,0.610604,0.987906
1,svr,0.218045,1.361141
8,adaboost,0.303116,1.369338
9,mlp,0.209858,1.457382
2,ridge,0.062252,1.526707
0,linear_reg,0.062252,1.526707


### Target Encoder

Target encoding, also known as mean encoding or likelihood encoding, is a technique used in machine learning for encoding categorical variables into numeric values based on the target variable.

1. It doens't give very good results for linear models compared to tree based but better than ordinal encodings.
2. It may leads to the problem of data leakage, therefore it is advisable to apply target encoding after train test split.
3. 

In [124]:
import category_encoders as ce

columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [125]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [126]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [127]:
scores.mean(),scores.std()

(0.829521918225536, 0.018384463379122907)

In [128]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [129]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [130]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [131]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [132]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.904798,0.447518
5,random forest,0.901470,0.449442
6,extra trees,0.901460,0.452774
7,gradient boosting,0.889032,0.508002
4,decision tree,0.827007,0.548404
9,mlp,0.855841,0.621116
8,adaboost,0.817330,0.687191
0,linear_reg,0.829522,0.713011
2,ridge,0.829536,0.713523
1,svr,0.782917,0.818851


# Observations:
1. Tree based mocdels Xgboost and RandomFforestClassifiers gave the best results.
2. OHE with target based encoding on sector column improved the results.
3. Now we will perform the hyper parameter tuning.

### Hyperparameter Tuning

In [133]:
from sklearn.model_selection import GridSearchCV

In [141]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [10, 20, 30],
    'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['sqrt', 'log2', None]  # Updated with valid options
}


In [142]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [143]:
# For RandomforestRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [144]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [145]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [146]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[CV 2/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.901 total time=   0.1s
[CV 1/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.889 total time=   0.1s
[CV 3/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.865 total time=   0.1s
[CV 6/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.849 total time=   0.1s
[CV 4/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.849 total time=   0.1s
[CV 8/10] END regressor__max_depth=10, regressor__max_features=sqrt, regressor__max_samples=0.1, regressor__n_estimators=50;, score=0.863 total time= 

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['property_type',
                                                                          'sector',
                                                                          'balcony',
                                                                          'agePossession',
                                                                          'furnishing_type',...
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['agePossession']),
                                                                        ('target_enc',
                                                                         TargetEncoder(),
                                                                         ['sector'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [10, 20, 30],
                         'regressor__max_features': ['sqrt', 'log2', None],
                         'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
                         'regressor__n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=4)

In [147]:
final_pipe = search.best_estimator_

In [148]:
search.best_params_

{'regressor__max_depth': 30,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [149]:
search.best_score_

0.903129115956613

In [59]:
final_pipe.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['sector', 'agePossession']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['sector'])])),
                ('regressor',
                 RandomForestRegressor(max_features='sqrt', max_samples=1.0,
                                       n_estimators=300))])

# XgBoostRegressor

In [ ]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.1],
    'regressor__subsample': [0.6, 0.8],
    'regressor__colsample_bytree': [0.6, 0.8],
    'regressor__gamma': [0, 0.1],
    'regressor__reg_alpha': [0, 0.1],
    'regressor__reg_lambda': [0, 0.1]
}


### Exporting the model

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [ ]:
pipeline.fit(X,y_transformed)

In [ ]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

### Trying out the predictions

In [ ]:
X

In [ ]:
X.columns

In [ ]:
X.iloc[0].values

In [ ]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df


In [ ]:
np.expm1(pipeline.predict(one_df))

In [ ]:
X.dtypes

In [ ]:
sorted(X['sector'].unique().tolist())